# Quickstart

Let's start by installing Rexify

In [ ]:
!pip install rexify

Get some data:

In [ ]:
!mkdir data
!curl --get https://storage.googleapis.com/roostr-ratings-matrices/rexify/completions.csv > data/events.csv

In [ ]:
import pandas as pd

In [ ]:
events = pd.read_csv('data/events.csv')
events

Next, we need to specify our schema:

In [ ]:
schema = {
    "user": {
        "account_id": "id",
    },
    "item": {
        "program_id": "id",
    },
    "context": {}
}

To preprocess our data, we can use the `FeatureExtractor`

In [ ]:
from rexify.features import FeatureExtractor

We just need to pass it the schema, and it's ready to roll out.

In [ ]:
feat = FeatureExtractor(schema=schema)

As a scikit-learn Transformer, it has two main methods: `.fit()` and `.transform()`. What `.fit_transform()` essentially does is: `.fit().transform()`.

During `.fit()`, it will take the schema, and infer what the preprocessing should look like - what transformations it should apply to the data before it's ready to be passed to the model. During `.transform()` it will apply those transformations, resulting in a `numpy.array` with the same number of rows as the original data.

In [ ]:
features = feat.fit_transform(events)
features

The `.make_dataset()` method converts the numpy array to a `tf.data.Dataset` with the format it's expecting.

In [ ]:
dataset = feat.make_dataset(features).batch(512)

We can now take our `Recommender` model and instantiate it.

During `.fit`, our `FeatureExtractor` also learns the right model parameters, so we don't need to worry about them. They're stored in the `model_params` property.

In [ ]:
from rexify.models import Recommender

In [ ]:
model = Recommender(**feat.model_params)

Being a `tensorflow.keras.Model` itself, in order to fit it, we need to first compile it:

In [ ]:
model.compile()

To fit it, all we need to do is pass our `tf.data.Dataset`:

In [ ]:
# model.fit(dataset)